# README
In this hackatron I tried to use LLM to do the feature engineer on the specialize topic which is the forest type classfication from Satellite Data

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "key"
os.environ["TAVILY_API_KEY"] = "key"
os.environ["LANGCHAIN_API_KEY"] = "key"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [81]:
from langchain import hub
from langchain_community.document_loaders import CSVLoader
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnablePick
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.beta.runnables.context import Context
from langchain_openai import OpenAIEmbeddings

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

# Agent

In [9]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI, OpenAI

In [11]:
import pandas as pd
df = pd.read_csv("../data/train.csv")
features = pd.read_csv("../data/IndexDB_scrape.csv")

In [14]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0, model="gpt-3.5-turbo-instruct"), df, verbose=True)

In [15]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [153]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
  [
    ("system",
     """You are a chatbot useful to give a code interpreter instructions to filter a df into a smaller table to answer the human question. 

Rule: Don't explain your code, just write the code using pandas.
(Most important) Rule2: no small talk or introduction needed. just the code.
Rule3: Your code must be one-liner execuable in python eval().
Rule4: Your code must create create new column in the df.
Example: Human: Wide Dynamic Range Vegetation.  
Your answer:df['WDRVI'] = (0.2 * (df['b8'] - df['b4'])) / (0.2 * (df['b8'] + df['b4']) + (1 - 0.2))
"""),
    ("user",
      """DF CONTEXT: You are working with Sentinel-2 satelite data. Here is the result of df.head() 
       id   b1   b11   b12   b2    b3   b4    b5    b6    b7    b8  b8_a    b9 nforest_type
0   2002  293  1927  1038  278   475  453   987  1773  2184  1900  2343  3039          MDF
1   3212  197  1598   697  201   347  228   682  1982  2449  2254  2685  2690          DDF
2  13312  929  1975  1031  982  1020  856  1220  2051  2421  2392  2671  2683          MDF
3  17020  132  1560   689  189   408  175   609  2117  2907  3024  3005  2955          MDF
4   5967  241  1944  1131  362   538  487   918  1549  1844  1702  2077  2043          MDF

        Question: {question}
        Calulation Formula:{formula}
        Related Variables: {variables}
        The names of the columns should be different from the list: {columns_list}
        Your Anser:
        """)
  ]
)

In [154]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
fomula_creation_prompt = ChatPromptTemplate.from_messages(
  [
    ("system",
     """You are Geo Scientist that work with Sentinel-2 Data. You are currently try to do the forest classfication using sentinel 2 data with the target class of Dry Evergreen Forest, Mixed deciduous forest, DECIDUOUS DIPTEROCARP FOREST. 
       Your job is to 1.write a formula that machine could understand. 2. give a short decription of what is that feature that formula represent and how that feature might or might not help model classify better.
Rule: When write the formula Don't explain your formula. just write a math formula by the data of sentinel 2.
(Most important) Rule2: When writing formula no small talk or introduction needed. just the formula.
"""),
    ("user",
      """
        Question: {question}
        """)
  ]
)

json_schema = {
    "title": "formula",
    "description": "formula as well as describtion.",
    "type": "object",
    "properties": {
        "formula": {
            "type": "string",
            "description": "formula of the question.",
        },
        "description": {
            "type": "string",
            "description": "short describtion about the what the feature represent.",
        }
    },
    "required": ["formula", "description"],
}

In [148]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0 , verbose=0)
formula_chain = (
    {"question": RunnablePassthrough()}
    | fomula_creation_prompt
    | llm.with_structured_output(json_schema)
)

In [123]:
formula_chain.invoke("create Adjusted transformed soil-adjusted VI")

{'formula': '(NIR - Red) / (NIR + Red + 0.5)',
 'description': 'Adjusted Transformed Soil-Adjusted Vegetation Index (ATSAVI) is a vegetation index that minimizes soil brightness influences by adjusting the traditional Soil-Adjusted Vegetation Index (SAVI) with a constant factor. It enhances the contrast between vegetation and soil, making it useful for vegetation classification and monitoring.'}

In [155]:
from operator import itemgetter
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0 , verbose=0)
rag_chain = (
    Context.setter("input")
    |RunnablePassthrough()|itemgetter("question")
    |formula_chain
    |Context.setter("formula_output")
    |{"formula": Context.getter("formula_output")|itemgetter("formula"), 
     "question": Context.getter("input")| itemgetter("question"), 
     "variables": Context.getter("input")| itemgetter("variables"), 
     "columns_list": Context.getter("input")| itemgetter("columns_list"),}
    | prompt
    | llm
    | StrOutputParser()
    | {
      "formula": RunnablePassthrough(),
      "description": Context.getter("formula_output")| itemgetter("description")
    }
)

In [150]:
response = rag_chain.invoke(input={"question":"create Adjusted transformed soil-adjusted VI", "variables": "", "columns_list":""})
response

{'formula': 'df[\'ATSAVI\'] = (df["b8"] - df["b4"]) / (df["b8"] + df["b4"] + 0.5)',
 'description': 'Adjusted Transformed Soil-Adjusted Vegetation Index (ATSAVI) represents the vegetation cover by considering the soil background reflectance. It helps in enhancing the contrast between vegetation and soil, making it useful for vegetation classification.'}

In [156]:
import re

def extract_column_name(s):
    # Define a regular expression pattern to match the column name
    pattern = r"df\['(.+?)'\]"  # Matches df['column_name']
    match = re.search(pattern, s)
    
    if match:
        column_name = match.group(1)  # Get the captured column name
        return column_name
    else:
        return None

In [160]:
columns_list = df.columns.to_list()
response_json = []

func_str = """
def feature_engineering(df):
"""

def add_function_line(original_string:str, new_string):
  return original_string + f"\n\t{new_string}"

for idx, row in features.iterrows():
  response = rag_chain.invoke(input={"question":"Create " + row["featurelink"], "variables": row["variables"], "columns_list": columns_list})
  func_str = add_function_line(func_str, f"#{row['featurelink']}") #add comment of variable full name
  func_str = add_function_line(func_str, response["formula"]) #add formula to the function
  response_json.append(response)
  columns_list.append(extract_column_name(response["formula"]))
  print(f"iteration:{idx} -> formula: {response['formula']}")


func_str = add_function_line(func_str, "return df")
  
  

iteration:0 -> formula: df['WDRVI'] = (df['b8'] - df['b4']) / (df['b8'] + df['b4'])
iteration:1 -> formula: df['WDRVI'] = (0.1 * df['b8_a'] - df['b4']) / (0.1 * df['b8_a'] + df['b4'])
iteration:2 -> formula: df['VARI_REDGE'] = (df['b8'] - df['b4']) / (df['b8'] + df['b4'])
iteration:3 -> formula: df['VARI_700'] = (df['b8'] - (2.5 * df['b4']) + 700) / (1 + df['b8'] + df['b4'])
iteration:4 -> formula: df['VARI_GREEN'] = (2.5 * (df['b8'] - df['b4'])) / ((df['b8'] + 6 * df['b4'] - 7.5 * df['b2']) + 1)
iteration:5 -> formula: df['VARI_700'] = (df['b8'] - df['b7']) / (df['b8'] + df['b7'])
iteration:6 -> formula: df['TCI'] = (df['b8'] - df['b4']) / (df['b8'] + df['b4'] + 0.1)
iteration:7 -> formula: df['Transformed_Vegetation_Index'] = (df['b8'] - df['b4']) / (df['b8'] + df['b4'] + 0.1)
iteration:8 -> formula: df['Transformed_Soil_Adjusted_Vegetation_Index_2'] = (2.4 * df['b8'] - 1.5 * df['b4']) / (df['b8'] + df['b4'] + 0.5)
iteration:9 -> formula: df['Transformed_Soil_Adjusted_Vegetation_Inde

In [ ]:
print(func_str)